# **PROJECT 1: MASS SHOOTINGS IN THE USA VISUALIZATION, USING ALTAIR AND STREAMLIT**
#### **Laura Humet and Maria Sans** - Information Visualization - GCED

# Data Cleaning and Data Aggregation Procedure Report

All these steps were applied to the 'dataset-crimes-usa' dataset:

1. **Column Removal**:
   - Removed the `Operations` column as all rows were defined as N/A.

2. **Handling Missing Values**:
   - Identified rows with missing values using:
     - `"Facet" > "Custom facet" > "Facet by blank"`
   - Deleted these rows instead of imputing values.

3. **Duplicate Verification**:
   - Verified and removed duplicates in the `Incident ID` column using:
     - `"Facet" > "Facet by duplicates"`

4. **Outlier and Numeric Verification**:
   - Ensured all numerical columns contained positive values:
     - `"Facet" > "Numeric Facet"`
   - Raised a query about potential outliers in `Victims Killed`.

  

5. **Text Transformation**:
   - Converted numerical fields (`Victims Killed`, `Victims Injured`, `Suspects Killed`, etc.) to numeric data types using the expression `value.toNumber()`.
   - Standardized date formatting by converting `Incident Date` to a date type using `value.toDate()`.

6. **Mass Edits**:
   - Standardized address values (e.g., merging variations of "Allegheny Ave and G St").
   - Trimmed whitespace in fields like `Incident ID`, `State`, `City Or County`, and `Address`.

7. **Derived Columns**:
   - Added `Incident Year` and `Incident Month` based on `Incident Date` for temporal analysis.
   - Added new column based on `City Or County` and `State` called `Direction`.
   - Created a `Geolocation` column based on `Direction`.
   - Appended "USA" to ensure geographic clarity in `Geolocation` and `Direction` fields.

8. **Fetching External Data**:
   - Created columns like `Latitude and Longitude` by querying geolocation services with formatted URLs.
   - Parsed geocoding API responses to extract latitude, longitude, county name, and FIPS codes.

9. **Column Splits**:
   - Split `Latitude and Longitude` into separate columns: `Latitude` and `Longitude`.

10. **Reconciliation with External Databases**:
   - Reconciled `State` and `City Or County` with external databases (e.g., Wikidata) to standardize place names.

11. **Row and Data Adjustments**:
   - Removed rows with blank or invalid latitude values.

12. **Final Enhancements**:
   - Added columns like `County Name` and `County FIPS` using Census API responses.

13. We found a **dataset called 'PopulationEstimates.xlsx'** from
 https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates

 from which we obtained the **population per State and per County**, and we merge it with our original dataset (dataset-crimes-usa) throughout the State column. This also could have been done with the reconcile method from OpenRefine.



These steps ensure the dataset is cleaned, consistent, and ready for analysis.

####**For the School Incidents Dataset** we have done almost the same procedure, so we don't think it's necessary to rewrite the steps

# Data Analysis and Visualization

In [48]:
import pandas as pd
import altair as alt
from vega_datasets import data
import numpy as np

In [49]:
!pip install altair==5.4.1

In [50]:
print(alt.__version__)

5.4.1


# Code for Data Preparation
We will use the next dataset for the first and second question:

In [51]:
source = pd.read_csv('Q1_final.csv')
source[['Latitude', 'Longitude']] = source['Latitude and Longitude'].str.split(',', expand=True)
source['Latitude'] = source['Latitude'].astype(float)
source['Longitude'] = source['Longitude'].astype(float)

# Extracting the state id, which is the first two numbers of its FIPS:
source['id'] = source['FIPS'].astype(str).str[:-3].astype(int)

# Extracting the keys we need for creating the charts:
info_states = source[['State', 'id', 'Population per State', 'Shootings Per State', 'Shootings per 100k Citizens']].drop_duplicates()

# Extracting the coordinates of each state by computing the mean of each county:
L = source.drop_duplicates().groupby('State').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
info_states = pd.merge(info_states, L, on='State', how='left')

# Sorting the DataFrame according to the 'State' column:
info_states = info_states.sort_values(by='State')

# Adding the missing coordinates --› [Hawaii, Maine, North Dakota]:
na_Latitude = ['19.741755', '45.367584', '47.650589']
na_Longitude = ['-155.844437', '-68.972168', '-100.437012']

idxs_nan = info_states[info_states['Latitude'].isna()].index

for i, index in enumerate(idxs_nan):
    info_states.loc[index, 'Latitude'] = na_Latitude[i]
    info_states.loc[index, 'Longitude'] = na_Longitude[i]

<ipython-input-51-794945f11c87>:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '19.741755' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  info_states.loc[index, 'Latitude'] = na_Latitude[i]
<ipython-input-51-794945f11c87>:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-155.844437' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  info_states.loc[index, 'Longitude'] = na_Longitude[i]


# **Q1: What are the states with large number of mass shootings per citizen?**

# Visualization Report: States with Mass Shootings per Citizen

## Bar Chart Design:
To visualize the number of mass shootings per citizen across states, we used a **Horizontal Bar Chart** for clear comparison.

1. **Visualization Decision**:
- We decided to create the bar chart horizontally to provide more space for text labels, since we wanted to provide the full name of the States. Furthermore, horizontal orientation can handle a large number of categories (in this case States) without becoming cluttered.

2. We ordered the bars in **decreasing order** to facilitate comparison between States and visualize the ranking.

3. **Refinements**:
   - Initially, the bars were color-coded based on shootings per 100,000 citizens. However, as the bar height inherently represents this data, we removed color to avoid redundancy and ensure clarity.
   - Tooltips and interactive elements (if applicable) were incorporated to improve user engagement.

The chart clearly shows that the **District of Columbia** has a disproportionately high rate of mass shootings compared to other states. This is a **disadvantage of bar charts**, since this longer bar dominates the visual space, making small diferences appear less significant.

Also, we decided not to plot all 50 States, to **avoid overcrowding**, which is another drawback of bar charts.

---

## Alternative Visualization, finally rejected:

An **alternative visualization** using a lollipop chart was considered but ultimately rejected. While it is visually appealing, it did not effectively represent the exact values and comparative differences as clearly as a bar chart. Additionally, lollipop charts can be less familiar to general audiences, reducing accessibility.




## Code for Data Preparation

In [52]:
shootings = pd.read_csv('Q1_final.csv')

## **Final Visualization**

In [79]:
# Extracting the 15 top states with more shootings per 100k citizen:
top_15_states = info_states.sort_values('Shootings per 100k Citizens', ascending=False).head(15)

# Creating the Bar Chart:
bar_chart_states = alt.Chart(top_15_states).mark_bar().encode(
    alt.X('Shootings per 100k Citizens:Q',
          title='Shootings per 100k Citizens',
    ),
    alt.Y('State:N', title='State', sort='-x'),
    color=alt.value('red')
).properties(
    title="Top 15 States with Highest Mass Shootings per 100k Citizens"
)


text = bar_chart_states.mark_text(
    align='left',
    baseline='middle',
    dx=5
).encode(
    text=alt.Text('Shootings per 100k Citizens:Q', format='.2f'),
    color=alt.value('grey')
)


Q1 = bar_chart_states + text
Q1

alt.LayerChart(...)

In [85]:
# Extracting the 15 top states with more shootings per 100k citizen:
top_15_states = info_states.sort_values('Shootings per 100k Citizens', ascending=False).head(15)

# Creating the Lollipop Chart:
lollipop_chart_states = alt.Chart(top_15_states).mark_rule().encode(
    alt.X('Shootings per 100k Citizens:Q',
          title='Shootings per 100k Citizens',
    ),
    alt.Y('State:N', title='State', sort='-x'),
    color=alt.value('red')
).properties(
    title="Top 15 States with Highest Mass Shootings per 100k Citizens"
)


lollipop_circles = alt.Chart(top_15_states).mark_circle(size=100).encode(
    alt.X('Shootings per 100k Citizens:Q'),
    alt.Y('State:N'),
    color=alt.value('red')
)

# Adding Text Labels:
text = lollipop_chart_states.mark_text(
    align='left',
    baseline='middle',
    dx=5
).encode(
    text=alt.Text('Shootings per 100k Citizens:Q', format='.2f'),
    color=alt.value('grey')
)


Q1 = lollipop_chart_states + lollipop_circles + text
Q1


alt.LayerChart(...)

# **Q2: How is the number of mass shootings per citizen distributed across the different counties in the US? And across states?**


# Visualization Report: Distribution of mass shootings across Counties and States

## Overview:
To represent mass shootings per citizen distributed across counties and states, we used cleaned datasets modified with OpenRefine. For counties, a separate dataset was prepared, while for states, we used the dataset processed earlier in the notebook.

## Visualization Design:
1. **Chosen Visualization**: **Choropleth Map**
   - This chart type effectively illustrates the **spatial distribution** of mass shootings, highlighting regional differences.
   - It allows users to identify patterns, such as whether mass shootings are concentrated in specific zones or spread across the country.
   - Also, familiarity with maps makes choropleths accessible to a broad audience.

2. **Challenges and Adjustments**:
   - **County-Level Map**:
     - Extracting the **county FIPS** was a major challenge. Initially, we attempted to merge the data with a supplementary dataset containing county FIPS codes, but many rows were lost during merging due to mismatched names.
     - Using OpenRefine, we successfully extracted the necessary FIPS codes and computed shootings per 100,000 citizens for each county.
   - **State-Level Map**:
     - For states, the FIPS codes were available using OpenRefine (reconcile option). We extracted the first two digits to match the map dataset IDs.
     - The visualization initially appeared skewed due to the **District of Columbia's** disproportionately high rate of mass shootings. As this state is very small and nearly imperceptible on the map, we decided to remove it. This improved the overall readability and proportionality of the chart.

---

## Drawbacks of Choropleth Maps:
- The shading in choropleth maps can misrepresent data because larger regions with lower values can appear more dominant than smaller regions with higher values due to area bias.
- Also, choropleths assume uniform data distribution within each geographic unit, which not may reflect reality.
- Choosing appropriate color scales is very important to minimize this problems. This is why we have used a redish scale of colors.

---


## Rejected Alternative: **Graduated Symbol Map**
- We also tested a **graduated symbol map**. To create this:
  - Coordinates of each county were extracted, and a mean of coordinates was calculated for counties within the same state.
  - Missing coordinates for **Hawaii**, **Maine**, and **North Dakota** were added manually.
- **Challenges**:
  - Since not every county has mass shootings, the computed mean coordinates did not align with the state’s center in many cases, making the map **aesthetically unpleasant**.
  - It was also difficult to **compare quantities** between states using this method.
- Considering these drawbacks, the graduated symbol map was discarded in favor of the **choropleth map**, which better represents the data.

---

## Final Outcome:
The **county-level choropleth map** reveals detailed patterns of shootings at a small level, while the **state-level map** provides a broader view. Together, they help to understand how mass shootings are distributed across the United States.



In [55]:
q2_source = pd.read_csv('Q2_dataset.csv')

q2 = q2_source[['State', 'County Name', 'County FIPS', 'population']]

# Adding a new column to count the mass shootings in each county and grouping the rows by the county's FIPS:
q2['Shootings_count'] = 1
shootings_count = q2.groupby('County FIPS').agg({'State':'first', 'County Name':'first', 'population':'first', 'Shootings_count':'count'}).reset_index()
shootings_count['Shootings_per_100k_citizens'] = shootings_count['Shootings_count']/shootings_count['population']*100
shootings_count = shootings_count.rename(columns={'County FIPS':'id'})

<ipython-input-55-f89748872c21>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2['Shootings_count'] = 1


In [80]:
### CLOROPLETH MAP TO REPRESENT THE MASS SHOOITNGS PER CITIZEN ACROSS THE COUNTIES ###

map_counties = alt.topo_feature(data.us_10m.url, 'counties')


map_foreground_counties = alt.Chart(map_counties).mark_geoshape(
    stroke='white'
).encode(
    color= alt.Color(
        'Shootings_per_100k_citizens:Q',
        scale=alt.Scale(scheme='reds', domain=[0, shootings_count['Shootings_per_100k_citizens'].max()]),
        legend=alt.Legend(title='Mass Shootings per 100k Citizens')
    ),
    tooltip=[alt.Tooltip('County Name:N', title='County Name'), alt.Tooltip('Shootings_per_100k_citizens:Q', title='Mass shootings')]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(shootings_count, 'id', ['County Name','Shootings_per_100k_citizens'])
).properties(
    title='Mass shootings per 100k citizens across US counties',
    width=500,
    height=300
).project(
    type='albersUsa'
)

map_background = alt.Chart(map_counties).mark_geoshape().encode(
    color=alt.value('white'),
    stroke=alt.value('lightgray')
).properties(
    title='Mass shootings per 100k citizens across US counties',
    width=500,
    height=300
).project(
    type='albersUsa'
)

Q2_counties = alt.layer(map_background, map_foreground_counties)

Q2_counties

alt.LayerChart(...)

In [82]:
### CLOROPLETH MAP TO REPRESENT THE MASS SHOOITNGS PER CITIZEN ACROSS THE STATES ###

info_states = info_states.drop(info_states[info_states['State']=='DISTRICT OF COLUMBIA'].index)

map_states = alt.topo_feature(data.us_10m.url, 'states')


Q2_states = alt.Chart(map_states).mark_geoshape().encode(
    color=alt.Color('Shootings per 100k Citizens:Q',
        scale=alt.Scale(scheme='reds', domain=[0, info_states['Shootings per 100k Citizens'].max()]),
        legend=alt.Legend(
            title='Mass Shootings per 100k Citizens',
            orient='right',
            titleFontSize=12,
            labelFontSize=10)
        ),
    stroke=alt.value('white'),
    tooltip=[alt.Tooltip('State:N', title='State'), alt.Tooltip('Shootings per 100k Citizens:Q', title='Mass Shootings per citizen')]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(info_states, 'id', ['State','Shootings per 100k Citizens']),
    default='0'
).properties(
    title='Mass Shootings per 100k Citizens across the US States',
    width=500,
    height=300
).project(
    type='albersUsa'
)

Q2_states

alt.Chart(...)

## Code for Alternative Visualizations (Further Rejected because there was a better solution)

In [86]:
# US states background:
background = alt.Chart(map_states).mark_geoshape(
    fill='lightpink',
    stroke='white'
).properties(
    title='Mass shootings per 100k citizens across US states',
    width=500,
    height=300
).project('albersUsa')

# Mass shootings grouped by state:
points_per_state = alt.Chart(info_states, title='Mass Shootings per 100k Citizens').mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size=alt.Size('Shootings per 100k Citizens:Q', title ='Mass Shootings per 100k Citizens'),
    color=alt.value('darkred'),
    tooltip=['State:N','Shootings per 100k Citizens:Q', 'Longitude:Q', 'Latitude:Q']
)

background + points_per_state

alt.LayerChart(...)

# **Q3: Are the mass shootings correlated with gun violence incidents in schools?**

# Investigating Correlation Between Mass Shootings and School Incidents: Visualization Design

## Chosen Visualization: **Bubble Chart**
To explore the potential correlation between mass shootings and gun violence incidents in schools, we chose a **scatter plot**, later enhanced into a **bubble chart**, as the most effective visualization for showing the **relationship between two continuous variables**.

## Design Process:
1. **Initial Concept**:
   - The first idea was to plot the **number of mass shootings and school incidents per state**.
   - However, this did not account for population differences between states.

2. **Final Approach**:
   - We plotted **mass shootings per million inhabitants** against **school incidents per million inhabitants**, ensuring a fair comparison across states.

3. **Enhancements for Clarity**:
   - Points were **color-coded** to represent different states.
   - Point size was adjusted based on state population, making it easier to identify states with larger populations.
   - A **regression line** was added to illustrate the overall trend and help users determine whether a positive correlation exists (which exists).

4. **Interactive Features**:
   - Added tooltips displaying:
     - The state name
     - Population size
     - Number of mass shootings
     - Number of school incidents
   - These features improve usability and provide additional context for each data point.

5. **Rationale for Bubble Chart**:
   - Bubble charts allow us **displaying three dimensions** of data (in our case number of mass shootings, number of school incidents, and population size).

   - They are **visually engaging** and quickly **draw attention to outliers**
  (like Ilinois or Louisiana) and **highlight correlation**, proving that an increase in mass shootings correlates with more school incidents.

- As a **disadvantage** Bubble Charts are **difficult to read** for large datasets as ours, since **overlapping bubbles can make the chart cluttered** and hard to interpret.
Also, the exact values are not easily discernible. This is why we have added a **tooltip**, to know the exact values of the three variables this bubble chart encodes, plus the name of the State each bubble encodes.



## Code for Data Preparation

In [59]:
school_incidents = pd.read_csv('School-incidents-csv.csv')
school_incidents['FIPS'] = school_incidents['FIPS'].astype('Int64')

mass_shootings = pd.read_csv('dataset-crimes-usa.csv')
mass_shootings['FIPS_State'] = mass_shootings['FIPS_State'].astype('Int64')

## **Final Visualization**

In [60]:
vermont_row = pd.DataFrame({
    'FIPS_State': [50],
    'Incident ID': [None],
    'Incident Date': [None],
    'Incident Month': [None],
    'Incident Year': [None],
    'State': ['Vermont'],
    'population': [647464],
    'City Or County': [None],
    'Direction': [None],
    'Latitude': [None],
    'Longitude': [None],
    'Address': [None],
    'Victims Killed': [0],
    'Victims Injured': [0],
    'Suspects Killed': [0],
    'Suspects Injured': [0],
    'Suspects Arrested': [0]
})

# Adding into the DataFrame of mass_shootings the state of Vermont:
mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)

# Grouping the schools shootings by the State FIPS:
school_counts = school_incidents.groupby('FIPS').size().reset_index(name='School Incidents')

# Grouping the total mass shootings by the State FIPS:
mass_shooting_counts = mass_shootings.groupby('FIPS_State').size().reset_index(name='Mass Shootings')

# Merging the two previous DataFrames:
merged_data = pd.merge(school_counts, mass_shooting_counts, left_on='FIPS', right_on='FIPS_State', how='inner')

# Adding the population of each State:
populations = mass_shootings[['FIPS_State', 'population', 'State']].drop_duplicates()
merged_data = pd.merge(merged_data, populations, left_on='FIPS', right_on='FIPS_State', how='left')

# Computing the proportion of mass shootings per million of citizens for each DataFrame:
merged_data['Mass Shootings per Million'] = (merged_data['Mass Shootings'] / merged_data['population']) * 1_000_000
merged_data['School Incidents per Million'] = (merged_data['School Incidents'] / merged_data['population']) * 1_000_000


### SCATTERPLOT TO OBSERVE THE CORRELATION BETWEEN MASS SHOOTINGS AND SCHOOL INCIDENTS ###

scatter_plot = alt.Chart(merged_data).mark_circle().encode(
    x=alt.X('Mass Shootings per Million', title='Mass Shootings per Million Inhabitants'),
    y=alt.Y('School Incidents per Million', title='School Incidents per Million Inhabitants'),
    size=alt.Size('population:Q', title='Population', legend=alt.Legend(orient='left')),
    color=alt.Color('State:N', title='State', legend=alt.Legend(orient='right', labelFontSize=10, symbolSize=15)),
    tooltip=['State', 'Mass Shootings', 'School Incidents', 'population',
             'Mass Shootings per Million', 'School Incidents per Million']
).properties(
    title='Correlation between Mass Shootings and School Incidents by State (per Million Inhabitants)'
)


regression_line = alt.Chart(merged_data).transform_regression(
    'Mass Shootings per Million', 'School Incidents per Million'
).mark_line(color='red').encode(
    x='Mass Shootings per Million:Q',
    y='School Incidents per Million:Q'
)


Q3 = (scatter_plot + regression_line).configure_mark(
    color='steelblue'
)

Q3

<ipython-input-60-514df35c6bed>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


alt.LayerChart(...)

## Code for Alternative Visualizations (Further Rejected because there was a better solution)

In [61]:
scatter_plot_rejected_1 = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Mass Shootings', title='Mass Shootings (per state, by FIPS)'),
    y=alt.Y('School Incidents', title='School Incidents (per state, by FIPS)'),
    tooltip=['FIPS', 'Mass Shootings', 'School Incidents']
).properties(
    title='Correlation between Mass Shootings and School Incidents by State (FIPS-based)'
).configure_mark(
    color='steelblue'
)

scatter_plot_rejected_1

alt.Chart(...)

In [62]:
# Counting the number of points (rows) in the merged data:
num_points = merged_data.shape[0]

# Printing the result:
print(f"Number of points in the scatter plot: {num_points}")

Number of points in the scatter plot: 50


In [63]:
# Getting the list of unique states (FIPS) in both datasets:
school_states = school_incidents['FIPS'].unique()
mass_shooting_states = mass_shootings['FIPS_State'].unique()

# Finding which states are missing in the merged data:
missing_in_school = set(mass_shooting_states) - set(school_states)
missing_in_mass_shooting = set(school_states) - set(mass_shooting_states)

print("States missing in school_incidents:", missing_in_school)
print("States missing in mass_shootings:", missing_in_mass_shooting)


States missing in school_incidents: {11}
States missing in mass_shootings: {<NA>}


Based on the output we can interpret the results as follows:


*   **States missing in school_incidents: {11}**
The state with FIPS code 11 is missing from the school_incidents. According to the FIPS code system this code (11) corresponds to **Washington D.C**.
*  **States missing in mass_shootings: {50, NA}**
The state with FIPS code 50 corresponds to Vermont. So it seems like Vermont is missing from the mass_shootings dataset.

Since Washington D.C is a federal district, it might not have school-related incidents recorded the same way as states.




Let's check if Vermont appears in the mass_shootings datset

In [64]:
# Checkign if Vermont (FIPS 50) appears in mass_shootings dataset:
vermont_data = mass_shootings[mass_shootings['FIPS_State'] == 50]
print(vermont_data)

     Incident ID Incident Date Incident Month Incident Year    State  \
3016        None          None           None          None  Vermont   

      FIPS_State  population City Or County Direction  Latitude  Longitude  \
3016          50    647464.0           None      None       NaN        NaN   

     Address  Victims Killed  Victims Injured  Suspects Killed  \
3016    None               0                0                0   

      Suspects Injured  Suspects Arrested  
3016                 0                  0  


The output indicates that there are no records in the mass_shootings dataset for Vermont confirming that there are simply no mass shootings recorded for Vermont in the dataset.

### We will add manually Vermont and create again the scatterplot

In [65]:
# Manually creating a row for Vermont (FIPS code 50) with 0 incidents:
vermont_row = pd.DataFrame({
    'FIPS_State': [50],
    'Incident ID': [None],
    'Incident Date': [None],
    'Incident Month': [None],
    'Incident Year': [None],
    'State': ['Vermont'],
    'population': [647464],
    'City Or County': [None],
    'Direction': [None],
    'Latitude': [None],
    'Longitude': [None],
    'Address': [None],
    'Victims Killed': [0],
    'Victims Injured': [0],
    'Suspects Killed': [0],
    'Suspects Injured': [0],
    'Suspects Arrested': [0]
})

# Appending the manually created row to the mass_shootings dataset:
mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)

school_counts = school_incidents.groupby('FIPS').size().reset_index(name='School Incidents')
mass_shooting_counts = mass_shootings.groupby('FIPS_State').size().reset_index(name='Mass Shootings')

merged_data = pd.merge(school_counts, mass_shooting_counts, left_on='FIPS', right_on='FIPS_State', how='inner')

# Creating the scatter plot again:
scatter_plot_rejected_2 = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Mass Shootings:Q', title='Mass Shootings per State'),
    y=alt.Y('School Incidents:Q', title='School Incidents per State'),
    tooltip=['State:N', 'Mass Shootings', 'School Incidents']

).properties(
    title='Correlation between Mass Shootings and School Incidents by State'
).configure_mark(
    color='steelblue'
)

scatter_plot_rejected_2


<ipython-input-65-eb6039bb45c5>:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


alt.Chart(...)

In [66]:
# Counting the number of points (rows) in the merged data:
num_points = merged_data.shape[0]

print(f"Number of points in the scatter plot: {num_points}")

Number of points in the scatter plot: 50


In [67]:
# Manually creating a row for Vermont (FIPS code 50) with 0 incidents:
vermont_row = pd.DataFrame({
    'FIPS_State': [50],
    'Incident ID': [None],
    'Incident Date': [None],
    'Incident Month': [None],
    'Incident Year': [None],
    'State': ['Vermont'],
    'population': [647464],
    'City Or County': [None],
    'Direction': [None],
    'Latitude': [None],
    'Longitude': [None],
    'Address': [None],
    'Victims Killed': [0],
    'Victims Injured': [0],
    'Suspects Killed': [0],
    'Suspects Injured': [0],
    'Suspects Arrested': [0]
})

mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


school_counts = school_incidents.groupby('FIPS').size().reset_index(name='School Incidents')
mass_shooting_counts = mass_shootings.groupby('FIPS_State').size().reset_index(name='Mass Shootings')


merged_data = pd.merge(school_counts, mass_shooting_counts, left_on='FIPS', right_on='FIPS_State', how='inner')


populations = mass_shootings[['FIPS_State', 'population', 'State']].drop_duplicates()
merged_data = pd.merge(merged_data, populations, left_on='FIPS', right_on='FIPS_State', how='left')

# Computing incidents per million of citizens:
merged_data['Mass Shootings per Million'] = (merged_data['Mass Shootings'] / merged_data['population']) * 1_000_000
merged_data['School Incidents per Million'] = (merged_data['School Incidents'] / merged_data['population']) * 1_000_000

# Creating the scatter plot:
scatter_plot_rejected_3 = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Mass Shootings per Million', title='Mass Shootings per Million Inhabitants'),
    y=alt.Y('School Incidents per Million', title='School Incidents per Million Inhabitants'),
    tooltip=['State', 'Mass Shootings', 'School Incidents', 'population',
             'Mass Shootings per Million', 'School Incidents per Million']
).properties(
    title='Correlation between Mass Shootings and School Incidents by State (per Million Inhabitants)'
).configure_mark(
    color='steelblue'
)

scatter_plot_rejected_3


<ipython-input-67-9e67109ab9fd>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


alt.Chart(...)

**It would be useful to add a Regression Line**

In [68]:
vermont_row = pd.DataFrame({
    'FIPS_State': [50],
    'Incident ID': [None],
    'Incident Date': [None],
    'Incident Month': [None],
    'Incident Year': [None],
    'State': ['Vermont'],
    'population': [647464],
    'City Or County': [None],
    'Direction': [None],
    'Latitude': [None],
    'Longitude': [None],
    'Address': [None],
    'Victims Killed': [0],
    'Victims Injured': [0],
    'Suspects Killed': [0],
    'Suspects Injured': [0],
    'Suspects Arrested': [0]
})


mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


school_counts = school_incidents.groupby('FIPS').size().reset_index(name='School Incidents')
mass_shooting_counts = mass_shootings.groupby('FIPS_State').size().reset_index(name='Mass Shootings')


merged_data = pd.merge(school_counts, mass_shooting_counts, left_on='FIPS', right_on='FIPS_State', how='inner')


populations = mass_shootings[['FIPS_State', 'population', 'State']].drop_duplicates()
merged_data = pd.merge(merged_data, populations, left_on='FIPS', right_on='FIPS_State', how='left')


merged_data['Mass Shootings per Million'] = (merged_data['Mass Shootings'] / merged_data['population']) * 1_000_000
merged_data['School Incidents per Million'] = (merged_data['School Incidents'] / merged_data['population']) * 1_000_000


scatter_plot = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Mass Shootings per Million', title='Mass Shootings per Million Inhabitants'),
    y=alt.Y('School Incidents per Million', title='School Incidents per Million Inhabitants'),
    tooltip=['State', 'Mass Shootings', 'School Incidents', 'population',
             'Mass Shootings per Million', 'School Incidents per Million']
).properties(
    title='Correlation between Mass Shootings and School Incidents by State (per Million Inhabitants)'
)


regression_line = alt.Chart(merged_data).transform_regression(
    'Mass Shootings per Million', 'School Incidents per Million'
).mark_line(color='red').encode(
    x='Mass Shootings per Million:Q',
    y='School Incidents per Million:Q'
)


Q3_rejected = (scatter_plot + regression_line).configure_mark(
    color='steelblue'
)

Q3_rejected


<ipython-input-68-52dc0cdf219d>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mass_shootings = pd.concat([mass_shootings, vermont_row], ignore_index=True)


alt.LayerChart(...)

# **Q4: How have mass shootings evolved the last years in the US?**

# Visualization Report: Evolution of Shootings in the USA (2019–2023)

## Chosen Visualization: **Line Chart**
To illustrate the evolution of shootings in the USA from 2019 to 2023, we selected a **Line Chart** as it effectively displays **trends over time** and allows viewers to observe fluctuations clearly.

## Design Enhancements:
1. **Annual Distinction**:
   - A discontinuity line was added at the end of each year, marking the transition to the next year.
   - This feature helps viewers focus on **annual patterns** and avoids ambiguity.

2. **Mean Reference Line**:
   - A **mean line** was added to represent the **average number of shootings** over the period.
   - This provides a baseline to evaluate which months and years had above or below average shootings.

3. **Highlighted Points**:
   - The **maximum** and **minimum** values of shootings during the period were marked to emphasize critical data points and draw attention to extreme variations.

---

## Rejected Alternatives:
1. **Separate Line Charts**:
   - Initially, we created 5 separate line charts, one for each year. While this simplified year-by-year analysis, it fragmented the data and made it harder to perceive long-term trends.

2. **Overlapping Line Charts**:
   - A single chart with multiple colored lines for each year was also considered. However, overlapping lines caused **visual confusion**, making it difficult to interpret the overall trend.

3. **Stacked Bar Chart**:
   - We experimented with a stacked bar chart where each bar represented a year, broken into segments for victims killed and injured.
   - Although this allowed for a compositional view of incidents, it failed to capture **monthly breakdowns** and **temporal evolution** across years.

---

## Final Choice Rationale:
The **Line Chart** provided the clearest representation of trends over time and it allows visualizing data changes at a glance. However, if many categories are compared in one line chart it messes the chart, as it happened when we tried to create different lines one per each year in the same chart.
Moreover, points along the line can be marked (as we did for the max and min values) to highlight specific data values.

Last but not least, line charts are very easy to read and understand for a wide audience.

## **Final visualization**

In [69]:
shootings['Incident Date'] = pd.to_datetime(shootings['Incident Date'])
filtered_data = shootings[(shootings['Incident Date'] >= '2019-01-01') & (shootings['Incident Date'] <= '2023-12-31')]
filtered_data['Year_Month'] = filtered_data['Incident Date'].dt.to_period('M')

monthly_counts = filtered_data.groupby('Year_Month').size().reset_index(name='Mass Shootings')

monthly_counts['Year_Month'] = monthly_counts['Year_Month'].astype(str)

# Extracting the minimum and maximum number of mass shootings per month to represent it in the chart:
monthly_counts['Value_Category'] = 'medium'
monthly_counts.loc[monthly_counts['Mass Shootings'].idxmax(), 'Value_Category'] = 'max'
monthly_counts.loc[monthly_counts['Mass Shootings'].idxmin(), 'Value_Category'] = 'min'

mean_shootings = monthly_counts['Mass Shootings'].mean()

<ipython-input-69-6780d97985d7>:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  filtered_data['Year_Month'] = filtered_data['Incident Date'].dt.to_period('M')


In [70]:
# Creating the line chart for mass shootings:
line_chart = alt.Chart(monthly_counts).mark_line(point=True, color='#D73027').encode(
    x=alt.X('Year_Month:T',
            title='Month-Year',
            axis=alt.Axis(
                labelAngle=45,
                tickCount=14,
                labelFontSize=10,
                labelPadding=10,
                format='%b %Y'
            )
    ),
    y=alt.Y('Mass Shootings:Q', title='Number of Mass Shootings'),
    tooltip=[alt.Tooltip('Year_Month:T', title='Month'), 'Mass Shootings']
).properties(
    title="Evolution of Mass Shootings in the US (2019-2023)",
    width=800,
    height=400,
)

# Highlighting the minimum and maximum number of mass shootings per month:
highlight_points = alt.Chart(monthly_counts).mark_point(filled=True).encode(
    x='Year_Month:T',
    y='Mass Shootings:Q',
    size=alt.Size(
        'Value_Category:N',
        scale=alt.Scale(
            domain=['min', 'max'],
            range=[200, 200]
        ),
        legend=None
    ),
    color=alt.Color(
        'Value_Category:N',
        scale=alt.Scale(
            domain=['min', 'max'],
            range=['red', 'red']
        ),
        legend=None
    ),
    tooltip=[
        alt.Tooltip('Year_Month:T', title='Month'),
        alt.Tooltip('Mass Shootings:Q', title='Shootings'),
        alt.Tooltip('Value_Category:N', title='Category')
    ]
)

# Writing 'min' and 'max' in the corresponding point:
text_labels = monthly_counts[monthly_counts['Value_Category'].isin(['min', 'max'])]

max_min_text = alt.Chart(text_labels).mark_text(
    align='center',
    dy=-20,
    fontSize=12,
    color='red'
).encode(
    x='Year_Month:T',
    y='Mass Shootings:Q',
    text='Value_Category:N'
)

# Mean line:
mean_line = alt.Chart(pd.DataFrame({'y': [mean_shootings]})).mark_rule(color='#D73027', size=2).encode(
    y='y:Q'
)

# Mean text label:
mean_text = alt.Chart(pd.DataFrame({'y': [mean_shootings], 'text': [f'Mean: {mean_shootings:.2f}']})).mark_text(
    align='left',
    dx=25,
    dy=-10,
    fontSize=12,
    color='#D73027'
).encode(
    y='y:Q',
    text='text:N'
)


year_starts = monthly_counts[monthly_counts['Year_Month'].str.endswith('01')].copy()
year_starts['x'] = year_starts['Year_Month']

# Discontinuity lines for each year:
discontinuity_lines = alt.Chart(year_starts).mark_rule(color='gray', strokeDash=[5, 5]).encode(
    x='x:T',
    size=alt.value(2)
)

Q4 = line_chart + highlight_points + mean_line + mean_text + discontinuity_lines + max_min_text


Q4

alt.LayerChart(...)

## Code for Alternative Visualizations (Further Rejected because there was a better solution)

In [71]:
import altair as alt
import pandas as pd


shootings['Incident Date'] = pd.to_datetime(shootings['Incident Date'])
filtered_data = shootings[(shootings['Incident Date'] >= '2019-01-01') & (shootings['Incident Date'] <= '2023-12-31')]
filtered_data['Year'] = filtered_data['Incident Date'].dt.year

charts = []

for year in range(2019, 2024):
    yearly_data = filtered_data[filtered_data['Year'] == year]
    yearly_data['Year_Month'] = yearly_data['Incident Date'].dt.to_period('M')
    monthly_counts = yearly_data.groupby('Year_Month').size().reset_index(name='Mass Shootings')
    monthly_counts['Year_Month'] = monthly_counts['Year_Month'].astype(str)

    mean_shootings = monthly_counts['Mass Shootings'].mean()

    line_chart = alt.Chart(monthly_counts).mark_line(color='red', point=True).encode(
        x=alt.X('Year_Month:T',
                title='Month-Year',
                axis=alt.Axis(
                    labelAngle=45,
                    tickCount=14,
                    labelFontSize=10,
                    labelPadding=10,
                    format='%b %Y'
                )
        ),
        y=alt.Y('Mass Shootings:Q', title='Number of Mass Shootings'),
        tooltip=['Year_Month', 'Mass Shootings']
    ).properties(
        title=f"Evolution of Mass Shootings in {year}",
        width=400,
        height=300
    )

    mean_line = alt.Chart(pd.DataFrame({'y': [mean_shootings]})).mark_rule(color='black', size=2).encode(
        y='y:Q'
    )

    mean_text = alt.Chart(pd.DataFrame({'y': [mean_shootings], 'text': [f'Mean: {mean_shootings:.2f}']})).mark_text(
        align='left',
        dx=25,
        dy=-10,
        fontSize=12,
        color='black'
    ).encode(
        y='y:Q',
        text='text:N'
    )

    yearly_chart = line_chart + mean_line + mean_text
    charts.append(yearly_chart)

# Creating a 2x2 grid layout by first horizontally concatenating two pairs of charts and then vertically concatenating the pairs
chart_1_2 = alt.hconcat(charts[0], charts[1])
chart_3_4 = alt.hconcat(charts[2], charts[3])

final_chart = alt.vconcat(chart_1_2, chart_3_4)

final_chart


<ipython-input-71-6e87efe55d9d>:13: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  yearly_data['Year_Month'] = yearly_data['Incident Date'].dt.to_period('M')
<ipython-input-71-6e87efe55d9d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_data['Year_Month'] = yearly_data['Incident Date'].dt.to_period('M')
<ipython-input-71-6e87efe55d9d>:13: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  yearly_data['Year_Month'] = yearly_data['Incident Date'].dt.to_period('M')
<ipython-input-71-6e87efe55d9d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

alt.VConcatChart(...)

In [72]:
shootings['Incident Date'] = pd.to_datetime(shootings['Incident Date'])


filtered_data = shootings[(shootings['Incident Date'] >= '2019-01-01') & (shootings['Incident Date'] <= '2023-12-31')]


filtered_data['Year'] = filtered_data['Incident Date'].dt.year
filtered_data['Month'] = filtered_data['Incident Date'].dt.month


monthly_counts = filtered_data.groupby(['Year', 'Month']).size().reset_index(name='Mass Shootings')


line_chart = alt.Chart(monthly_counts).mark_line(point=True).encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0, labelFontSize=10)),
    y=alt.Y('Mass Shootings:Q', title='Number of Mass Shootings'),
    color=alt.Color('Year:N', title='Year'),
    tooltip=['Year', 'Month', 'Mass Shootings']
).properties(
    title="Monthly Evolution of Mass Shootings (2019-2023)",
    width=800,
    height=400
)


mean_shootings = monthly_counts['Mass Shootings'].mean()
mean_line = alt.Chart(pd.DataFrame({'y': [mean_shootings]})).mark_rule(color='black', size=2).encode(
    y='y:Q'
)


mean_text = alt.Chart(pd.DataFrame({'y': [mean_shootings], 'text': [f'Mean: {mean_shootings:.2f}']})).mark_text(
    align='left',
    dx=10,
    dy=-10,
    fontSize=12,
    color='black'
).encode(
    y='y:Q',
    text='text:N'
)


Q4_rejected = line_chart + mean_line + mean_text
Q4_rejected


alt.LayerChart(...)

In [73]:
yearly_victims = shootings.groupby('Incident Year').agg(
    Victims_Killed=('Victims Killed', 'sum'),
    Victims_Injured=('Victims Injured', 'sum')
).reset_index()


victims_melted = yearly_victims.melt(id_vars=['Incident Year'],
                                     value_vars=['Victims_Killed', 'Victims_Injured'],
                                     var_name='Victim Type', value_name='Count')


stacked_bar_rejected = alt.Chart(victims_melted).mark_bar().encode(
    x=alt.X('Incident Year:O', title='Year'),
    y=alt.Y('Count:Q', title='Number of Victims'),
    color=alt.Color('Victim Type:N', scale=alt.Scale(scheme='set1'), title='Type of Victims'),
    tooltip=['Incident Year', 'Victim Type', 'Count']
).properties(
    title="Victims of Mass Shootings by Year"
)

stacked_bar_rejected


alt.Chart(...)



---



# **Extra Additional Visualization**

In [74]:
df = pd.read_csv('Q2_dataset.csv')
df.head()

,Incident ID,Incident Date,Incident Month,Incident Year,State,FIPS_State,population,City Or County,Direction,json,...,Longitude,census,County FIPS,County Name,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested
0,2201535,2021-12-31T00:00:00Z,12,2021,Maryland,24,6177224.0,Capitol Heights,"Capitol Heights, Maryland, USA","[{""place_id"":320358147,""licence"":""Data © OpenS...",...,-76.913477,"{""result"":{""geographies"":{""State Legislative D...",24033,Prince George's County,Cindy Ln,0,4,0,0,0
1,2201716,2021-12-31T00:00:00Z,12,2021,Mississippi,28,2961279.0,Gulfport,"Gulfport, Mississippi, USA","[{""place_id"":279519744,""licence"":""Data © OpenS...",...,-89.092816,"{""result"":{""geographies"":{""State Legislative D...",28047,Harrison County,1200 block of Lewis Ave,4,3,0,0,6
2,2201216,2021-12-31T00:00:00Z,12,2021,California,6,39538223.0,Los Angeles,"Los Angeles, California, USA","[{""place_id"":293793093,""licence"":""Data © OpenS...",...,-118.242766,"{""result"":{""geographies"":{""State Legislative D...",6037,Los Angeles County,10211 S. Avalon Blvd,0,6,0,0,0
3,2200968,2021-12-30T00:00:00Z,12,2021,Pennsylvania,42,13002700.0,Philadelphia,"Philadelphia, Pennsylvania, USA","[{""place_id"":331603944,""licence"":""Data © OpenS...",...,-75.163526,"{""result"":{""geographies"":{""State Legislative D...",42101,Philadelphia County,5100 block of Germantown Ave,0,6,0,1,1
4,2201052,2021-12-30T00:00:00Z,12,2021,Missouri,29,6154913.0,Kirksville,"Kirksville, Missouri, USA","[{""place_id"":344984709,""licence"":""Data © OpenS...",...,-92.583394,"{""result"":{""geographies"":{""State Legislative D...",29001,Adair County,700 block of E Dodson St,3,1,0,0,2


Let's have a look at the data to see what we can extract from **Police Brutality**

In [77]:
df['Incident Date'] = pd.to_datetime(df['Incident Date'])


total_suspects_killed = df['Suspects Killed'].sum()
total_suspects_injured = df['Suspects Injured'].sum()

# Police brutality by year (total suspects killed/injured per year):
by_year = df.groupby('Incident Year').agg(
    total_suspects_killed=('Suspects Killed', 'sum'),
    total_suspects_injured=('Suspects Injured', 'sum')
).reset_index()

# Police brutality by state (total suspects killed/injured per state):
by_state = df.groupby('State').agg(
    total_suspects_killed=('Suspects Killed', 'sum'),
    total_suspects_injured=('Suspects Injured', 'sum')
).reset_index()


print(f"Total Suspects Killed: {total_suspects_killed}")
print(f"Total Suspects Injured: {total_suspects_injured}")
print("\nPolice Brutality by Year:\n", by_year)
print("\nPolice Brutality by State:\n", by_state)


Total Suspects Killed: 170
Total Suspects Injured: 135

Police Brutality by Year:
    Incident Year  total_suspects_killed  total_suspects_injured
0           2019                     33                      11
1           2020                     22                      14
2           2021                     38                      37
3           2022                     36                      39
4           2023                     41                      34

Police Brutality by State:
                    State  total_suspects_killed  total_suspects_injured
0                Alabama                      2                       6
1                 Alaska                      0                       0
2                Arizona                      5                       1
3               Arkansas                      1                       4
4             California                     17                       5
5               Colorado                      4                       6


In [78]:
import altair as alt
import pandas as pd


by_year = df.groupby('Incident Year').agg(
    total_suspects_killed=('Suspects Killed', 'sum'),
    total_suspects_injured=('Suspects Injured', 'sum')
).reset_index()


by_year_long = by_year.melt(
    id_vars=['Incident Year'],
    value_vars=['total_suspects_killed', 'total_suspects_injured'],
    var_name='Category',
    value_name='Count'
)

# Creating the grouped bar chart
grouped_bar_chart = alt.Chart(by_year_long).mark_bar(size=30).encode(
    x=alt.X('Incident Year:O', title='Year'),
    xOffset=alt.XOffset('Category:N'),
    y=alt.Y('Count:Q', title='Number of Suspects'),
    color=alt.Color(
        'Category:N',
        title='Category',
        scale=alt.Scale(
            domain=['total_suspects_killed', 'total_suspects_injured'],
            range=['#c0021e', '#ff8a5e']
        )
    ),
    tooltip=[
        alt.Tooltip('Incident Year:O', title='Year'),
        alt.Tooltip('Count:Q', title='Count'),
        alt.Tooltip('Category:N', title='Category')
    ]
).properties(
    title='Total Suspects Killed and Injured by Year',
    width=600,
    height=400
)


extra_chart_data = df.groupby(['FIPS_State']).agg({
    'State':'first',
    'Suspects Killed':'sum',
    'Suspects Injured':'sum',
    'Suspects Arrested':'sum'
}).reset_index()

# Creating the map of total suspects killed by state
map_states = alt.topo_feature(data.us_10m.url, 'states')

additional_map = alt.Chart(map_states).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color(
        'Suspects Killed:Q',
        scale=alt.Scale(scheme='reds', domain=[0, extra_chart_data['Suspects Killed'].max()]),
        legend=alt.Legend(title='Total Suspects Killed')
    ),
    tooltip=[
        alt.Tooltip('State:N', title='State Name'),
        alt.Tooltip('Suspects Killed:Q', title='Total Suspects Killed')
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(extra_chart_data, 'FIPS_State', ['Suspects Killed', 'State'])
).properties(
    title='Total Suspects Killed by Police across U.S. States - POLICE BRUTALITY',
    width=600,
    height=400
).project(
    type='albersUsa'
)


final_chart = alt.vconcat(
    alt.hconcat(grouped_bar_chart),
    additional_map
).configure_axis(
    grid=False
)

final_chart


alt.VConcatChart(...)

Finally, we created two additional visualizations to explore police brutality: a paired bar chart to track trends over the years and a choropleth map to examine differences across states.

The **paired bar chart** compares the total number of suspects killed versus injured each year in the USA. We organized data by year, using two bars per year to represent injuries and fatalities. To reduce clutter, we used distinct colors for killed and injured and added clear labels and a legend for better readibility.

The chart compares how often suspects end up injured versus, well... not walking away at all.

**Alternatives**: We thought of creating a stacked bar chart but didn't work as well in comparing both variables.

---


As for the **choropleth map**, we only focus on the number of suspects killed and we distribute the data across States, to see which States have more aggressive police presence - and by extension, where it might not be the best idea to engage in a shooting.


#Conclusion of the project

Overall, this project has allowed us to apply the visualization techniques we've learned in a real-world context. We've noticed that visualizing data isn't just about making charts. Instead, it's about telling a story, and, making complex issues (as it is mass shootings in the usa) more understandable for the audience.
